In [1]:
# =============================================================================
# Biblioteca
# =============================================================================
import numpy as np
import pandas as pd
import ipywidgets as ipw

from IPython.display import display
from IPython.display import Markdown
from IPython.display import HTML
import IPython.display as ipy
from datetime import datetime
import time
import pickle
import sys
from IPython.display import clear_output #habilitar o limpar ipw.output
from ipyfilechooser import FileChooser #alterativa ao widget.file_uload que não fornece initial directory


#         return ipw.HTML([Markdown(f"<span style='{estilo}'>{texto}</span>")])
# import matplotlib.pyplot as plt
# from ipywidgets import interact, fixed, interact_manual
### =============================================================================
### Classes
### =============================================================================
#Keys
# sys.path.insert(1, '..')
sys.path.insert(1, '../wfuncoes py')
import keys as k

## Função Markdown

def md(Texto):
    display(ipy.Markdown(Texto))

#Hemera
from cls.hemera import Hemera

# #Hemera
# try:
#     del sys.modules["hemera"]
#     import hemera
    
# except:
#     pass
# import hemera

global h1
h1 = Hemera()

In [2]:
##====================================================
##====================================================
# CABEÇALHO
##====================================================
##====================================================
## LAYOUT
##====================================================
# CABEÇALHO
lbl_hemera_load = ipw.HTML(f"<div style='font-weight: bold; background-color: lightgreen;'>Hemera carregado:</div>")
# lbl_hemera_load = md('Hemera carregado:', cor='gray',cor_fundo='lightgreen',tamanho=20, modo='Box')

txt_hemera_load = ipw.Text(
    value= '(vazio)',
    placeholder='Type something',
    layout=ipw.Layout(width='150px'),
    disabled=False   
)
if h1.dt_update.strftime('%Y-%m-%d_%H_%M_%S') != '0001-01-01_00_00_00':
    txt_hemera_load.value = h1.dt_update.strftime('%d/%m/%Y %H:%M:%S')

spacer = ipw.HTML("<div style='width: 40px;'></div>")

lbl_sap_load = ipw.HTML(f"<div style='font-weight: bold; background-color: lightgreen;'>SAP carregado:</div>")
txt_sap_load = ipw.Text(
    value= '(vazio)',
    placeholder='Type something',
    layout=ipw.Layout(width='150px'),
    disabled=False   
)

file_hemera = FileChooser('./pickle/rscm')
file_hemera.filter_pattern = '*.pkl'

# out_head = ipw.Output()

##====================================================
## FUNÇÕES
##====================================================
def change_title(b):
#     if log_hemera.value: 
#     with out_head:
    h1._log = False
    h1.carregar(file_hemera.selected_filename[-23:-4])
    txt_hemera_load.value = h1.dt_update.strftime('%d/%m/%Y %H:%M:%S')

##====================================================
## GATILHOS
##====================================================
file_hemera.register_callback(change_title)

##====================================================
## GUI
##====================================================
linha_horizontal = ipw.HTML("<hr style='border: 1px solid black;'>")

smc_load_box = ipw.VBox([ipw.HBox([lbl_hemera_load,txt_hemera_load,spacer,spacer,spacer,spacer,spacer,lbl_sap_load,txt_sap_load]),
                        ipw.HBox([file_hemera,spacer,spacer, spacer,spacer,spacer,spacer])])

display(smc_load_box, linha_horizontal)


##====================================================
##====================================================
# TAB1
##====================================================
##====================================================
## LAYOUT
##====================================================
# HEMERA
options = ['Extrair', 'Transformar', 'Carregar']
tgg_hemera = ipw.SelectMultiple(
    options=options,
    description="ETL RSCM: ",
    rows=len(options),
    layout=ipw.Layout(width='200px'),
    value=['Extrair', 'Transformar', 'Carregar'],
    tooltips=['Extrair relatório rscm do Hemera', 'Transformar o relatório rscm', 'Carregar Dataframe recém baixado']
)
btn_hemera = ipw.Button(description="Executar")
out_hemera = ipw.Output()
log_hemera = ipw.Checkbox(
    value=True,
    description='exibir log',
    disabled=False,
    indent=False)
acesso_inicial = True

# Tela de login
user_input = ipw.Text(description='Usuário:')
password_input = ipw.Password(description='Senha:')
login_button = ipw.Button(description='Login')
user_input.value = k.user_rede_clb
password_input.value = k.pass_rede

# LIMPAR LOG
btn_limpar = ipw.Button(description='Limpar logs')

##====================================================
## FUNÇÕES
##====================================================
# Executar
def executar_clicked_on(button):
    global acesso_inicial
    h1._log = log_hemera.value
    limpar_output(btn_limpar)
    if tgg_hemera.value[0] == 'Extrair' and acesso_inicial:
        acesso_inicial = False
        login_box.layout.display = 'flex'
    else:
        login_clicked_on(None)

# Login
def login_clicked_on(event):
    h1._log = log_hemera.value
    limpar_output(btn_limpar)
    for i in tgg_hemera.value:
        if i == 'Extrair':
            login_box.layout.display = 'none'
            with out_hemera:
                h1.extrair(user_input.value, password_input.value)
        if i == 'Transformar':
            with out_hemera:
                h1.transformar()
        if i == 'Carregar':
            with out_hemera:
                #if h1.dt_update == datetime(year=1, month=1, day=1, hour=0, minute=0, second=0):
                #    print('Erro: Nenhum arquivo extraído e transformado para carregar.')
                #else:
                h1.carregar() #h1.dt_update.strftime('%Y-%m-%d_%H_%M_%S'))
                txt_hemera_load.value = h1.dt_update.strftime('%d/%m/%Y %H:%M:%S')
                display(h1.dt_update)
                display(h1.tb)
    with out_hemera:
        display('Finalizado com sucesso.')

# Limpar log
def limpar_output(button):
    with out_hemera:
        clear_output()

##====================================================
## GATILHOS
##====================================================
login_button.on_click(login_clicked_on)
btn_hemera.on_click(executar_clicked_on)
btn_limpar.on_click(limpar_output)

##====================================================
## GUI
##====================================================
# Criar a janela padrão
login_box = ipw.VBox([user_input, password_input, login_button])

# Exibição
hemera_box = ipw.VBox([tgg_hemera, log_hemera, ipw.HBox([btn_hemera, btn_limpar]),login_box, out_hemera])
login_box.layout.display = 'none'


##====================================================
##====================================================
# TAB2
##====================================================
##====================================================
## SAP_CLIENTES
##====================================================

file_name = './pickle/sap_clientes/sap_clientes.pkl'
df = pd.read_pickle(file_name)
# display(df.columns)
# display(df)

# file_name = './input/sap_clientes.csv'
# df_result = pd.read_csv(file_name,sep=';'
#               ,names = ['instalacao','inst_dt_ligacao','inst_dt_deslig','medidor','med_dt_desde','med_dt_ate','cc','med_material','med_equi_log']
#               ,dtype = {'instalacao': 'int64','inst_dt_ligacao': 'str','inst_dt_deslig': 'str','medidor': 'int64','med_dt_desde': 'str','med_dt_ate': 'str','cc': 'int64','med_material': 'int64','med_equi_log': 'str'}
#               ,parse_dates = ['inst_dt_ligacao','inst_dt_deslig','med_dt_desde','med_dt_ate']
#               ,engine = 'python'
#               )
# df_result.inst_dt_ligacao = df_result.inst_dt_ligacao.dt.strftime('%Y-%m-%d')
# display(df_result)

##====================================================
##====================================================
# TAB3
##====================================================
##====================================================
## SMC
##====================================================

##====================================================
##====================================================
# TAB4
##====================================================
##====================================================
## NOTAS
##====================================================

##====================================================
##====================================================
## GUI PRINCIPAL
##====================================================
##====================================================

names = ['ETL Hemera', 'ETL SAP', 'Relatórios','Faturamento', 'Notas', 'Manutenção']
construcao = ipw.HTML(f"<div style='font-weight: bold; background-color: red;'>Em construção!</div>")

tab = ipw.Tab()
[tab.set_title(i, title) for i, title in enumerate(names)]
tab.children = [hemera_box, construcao, construcao,construcao,construcao,construcao]

# Crie um widget VBox para conter o Tab
vbox = ipw.VBox([tab])

# Exiba o VBox
display(vbox)

HTML(value="<hr style='border: 1px solid black;'>")

In [3]:
btn_html = HTML(
'''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Modo dash / code"></form>''')

md('**WSMC**')
display(btn_html)

**WSMC**